In [ ]:
# import python libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, tqdm # import utility libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input

from tensorflow.keras.applications import VGG19
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.layers import LSTM, GRU, Conv2D, MaxPooling2D, Dense, Activation, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, plot_confusion_matrix

In [ ]:

for dirname, _, filenames in os.walk('path for images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# directory with training covid pictures
train_covid_dir = os.path.join('Path to/CT_COVID')

# Directory with training human pictures
train_non_covid_dir = os.path.join('Path to/CT_NonCOVID')

print(train_covid_dir)
print(train_non_covid_dir)

In [ ]:
# print images of covid images
train_covid_names = os.listdir(train_covid_dir)
print(f'TRAIN SET COVID: {train_covid_names[:10]}')

print('')

# print names of non covid images
train_non_covid_names = os.listdir(train_non_covid_dir)
print(f'TRAIN SET NON-COVID: {train_non_covid_names[:10]}')

In [ ]:
# printing total number of images in each directory
print(f'total training covid images: {len(os.listdir(train_covid_dir))}')
print(f'total training non-covid images: {len(os.listdir(train_non_covid_dir))}')

covid = len(os.listdir(train_covid_dir))
non_covid = len(os.listdir(train_non_covid_dir))
print('Total CT-scanned images in dataset: %s' % str(covid + non_covid))

In [ ]:

# parameters for our plot; we will output images in a 4x4 configuration
nrows, ncols = 4, 4

# index for iterating over images
pic_index = 0

# setting up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_normal_pix = [os.path.join(train_covid_dir, fname) 
                for fname in train_covid_names[pic_index-8:pic_index]]
next_malig_pix = [os.path.join(train_non_covid_dir, fname) 
                for fname in train_non_covid_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_normal_pix + next_malig_pix):
    
  # setting up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


In [ ]:
data_dir = 'Path to training data'
X, Y = 224, 224
BATCH_SIZE = 32

# setting up the data generator for train and validation splits
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest'
                            )

# test_datagen = ImageDataGenerator(rescale=1/255.0)

training_data = datagen.flow_from_directory(data_dir,
                                    class_mode = "binary",
                                    target_size = (X, Y),
                                    color_mode="rgb",
                                    batch_size = BATCH_SIZE, 
                                    shuffle = False,
                                    subset='training',
                                    seed = 42
                                    )

validation_data = datagen.flow_from_directory(data_dir,
                                      class_mode = "binary",
                                      target_size = (X, Y),
                                      color_mode="rgb",
                                      batch_size = BATCH_SIZE, 
                                      shuffle = False,
                                      subset='validation',
                                      seed = 42)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def nn_model():
    '''
    A function to make a deep convolutional neural network with convolutional, pooling and 
    fully-connected layers
    '''
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), padding='same', activation=tf.nn.relu, 
                     input_shape=(X, Y, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu))
    model.add(MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(0.4))

    model.add(Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])
    
    return model

model = nn_model()
model

In [ ]:
model.summary()

In [ ]:
# setting up early stopping to stop training when max acc is reached
# we are using validation loss as parameter for monitoring the training process

# setup an early stopping object for monitoring model performance
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=3)

# setting up the csv logger
logger = CSVLogger('logs.csv', append=True)

# fitting the model to training data and validating on validation data
history = model.fit(
    training_data,
    validation_data=validation_data,
    epochs=10,
    callbacks=[early_stopping, logger]
)

In [ ]:
model = Sequential()
vgg19 = VGG19(input_shape=(224, 224, 3),
                      weights='imagenet',
                      include_top=False)

# swtting traimable to false
for layer in vgg19.layers:
    layer.trainable=False

In [ ]:
x = Flatten()(vgg19.output)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)
model

In [ ]:

x = GlobalAveragePooling2D()(vgg19.output)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
y = Dense(1, activation='sigmoid')(x)

model_2 = Model(inputs=vgg19.input, outputs=y)
model_2

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])

In [ ]:
# setting up early stopping to stop training when max acc is reached
# we are using validation loss as parameter for monitoring the training process

# fitting the model to training data and validating on validation data
history = model.fit(
    training_data,
    validation_data=validation_data,
    epochs=60,
    callbacks=[logger]
)

In [ ]:
hist = history.history
plt.style.use('_classic_test_patch')

epochs = range(1, len(hist['loss']) + 1)
acc = hist['accuracy']
loss = hist['loss']
val_acc = hist['val_accuracy']
val_loss = hist['val_loss']

# plot loss versus accuracy
plt.figure(figsize=(10, 7))
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(epochs, acc, label='accuracy')
plt.plot(epochs, val_acc, label='val_acc')
plt.legend(loc='lower right')

# plot validation versus accuract
plt.figure(figsize=(10, 7))
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(epochs, loss, label='loss', color='g')
plt.plot(epochs, val_loss, label='val_loss', color='r')
plt.legend()

In [ ]:
import json

In [ ]:

model_json = model.to_json()
with open("modelOne.json", "w") as json_file:
    json_file.write(model_json)
model.save("path to save the trained model/modelOne.h5")
print("Model Saved to the disk")

In [ ]:
model = keras.models.load_model("path to load the /modelOne.h5")
model.summary()

In [ ]:

import cv2

image = cv2.imread("Path to read test image")
print(type(image))
image = cv2.resize(image, (224, 224))
print(type(image))



In [ ]:
# Make predictions
predictions = model.predict(image.reshape(1, 224, 224, 3))

# Print the predictions
print(predictions)